In [1]:
import os
import sys

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
print(os.path.join(current_working_directory, ".."))
sys.path.append(os.path.join(current_working_directory, ".."))

/Users/L024258/lilly_work/github-copilot/exploration/notebooks/..


In [2]:
from decouple import AutoConfig
config = AutoConfig(search_path='./../.env')

os.environ["AZURE_OPENAI_API_KEY"] = config('OPENAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = config('AZURE_ENDPOINT')

In [3]:
from langchain_openai import AzureChatOpenAI

In [4]:
from typing import Any


class Agent():
    def __init__(self, sys_msg=""):
        self.sys_msg = sys_msg
        self.model = AzureChatOpenAI(
                openai_api_version=config('AZURE_CHAT_OPENAI_API_VERSION'),
                azure_deployment=config('AZURE_GPT4_CHAT_OPENAI_DEPLOYMENT'),
                temperature=0,
                max_tokens=1024
            )
        self.messages = []
        if sys_msg:
            self.messages.append({"role": "system", "content": sys_msg})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = self.model.invoke(self.messages)
        return completion.content
        

In [5]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate Simple Interest:
e.g. calculate_si: 1000, 5, 10
Runs calculate_si and returns the simple interest

calculate Compound Interest:
e.g. calculate_ci: 1000, 5, 10
Runs calculate_ci and returns the compound interest

Example session:

Question: What is the simple interest on a loan of 1000 INR at 5% for 10 years?
Thought: I should calculate the simple interest using calculate_si function
Action: calculate_si: 1000, 5, 10
PAUSE

You will be called again with this:

Observation: 500

You then output:

Answer: Simple interest is 500 INR.
""".strip()

In [6]:
def calculate_si(p, r, t):
    return (p * r * t) / 100

def calculate_ci(p, r, t):
    return p * (1 + r / 100) ** t - p

known_actions = {
    "calculate_si": calculate_si,
    "calculate_ci": calculate_ci
}

#### Example 1

In [7]:
cbot = Agent(prompt)

In [8]:
result = cbot("What is the simple interest on a loan of 1000 USD at 5% for 10 years?")
print(result)

Thought: I should calculate the simple interest using calculate_si function
Action: calculate_si: 1000, 5, 10
PAUSE


In [9]:
result = calculate_si(1000, 5, 10)
print(result)

500.0


In [10]:
next_prompt = "Observation: {}".format(result)
cbot(next_prompt)

'Answer: The simple interest is 500.0 USD.'

In [11]:
cbot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate Simple Interest:\ne.g. calculate_si: 1000, 5, 10\nRuns calculate_si and returns the simple interest\n\ncalculate Compound Interest:\ne.g. calculate_ci: 1000, 5, 10\nRuns calculate_ci and returns the compound interest\n\nExample session:\n\nQuestion: What is the simple interest on a loan of 1000 INR at 5% for 10 years?\nThought: I should calculate the simple interest using calculate_si function\nAction: calculate_si: 1000, 5, 10\nPAUSE\n\nYou will be called again with this:\n\nObservation: 500\n\nYou then output:\n\nAnswer: Simple interest is 500 INR.'},
 {'role': 'user',
  'content': 'What is the si

#### Example 2

In [12]:
cbot = Agent(prompt)

In [13]:
question = "What are the simple and compound interest on a loan of 1000 USD at 5% for 10 years?"

In [14]:
response = cbot(question)
print(response)

Thought: I should calculate the simple interest first using calculate_si function and then calculate the compound interest using calculate_ci function.
Action: calculate_si: 1000, 5, 10
PAUSE


In [15]:
next_prompt = "Observation: {}".format(calculate_si(1000, 5, 10))
print(next_prompt)

Observation: 500.0


In [16]:
response = cbot(next_prompt)
print(response)

Thought: Now that I have calculated the simple interest, I should calculate the compound interest using calculate_ci function.
Action: calculate_ci: 1000, 5, 10
PAUSE


In [17]:
next_prompt = "Observation: {}".format(calculate_ci(1000, 5, 10))
print(next_prompt)

Observation: 628.894626777442


In [18]:
response = cbot(next_prompt)
print(response)

Answer: The simple interest is 500.0 USD and the compound interest is approximately 628.89 USD.


### Lopping the Agent Exeution

In [37]:
import re
action_re = re.compile('^Action: (\w+): (.*)$')
answer_re = re.compile('^Answer: (.*)$')

In [41]:
def query_cbot(question):
    cbot = Agent(prompt)
    next_prompt = question
    while True:
        response = cbot(next_prompt)
        print(response)
        if answer_re.match(response):
            print("Execution Complete")
            return
        actions = [action_re.match(r) for r in response.split('\n') if action_re.match(r)]
        if actions:
            action, args = actions[0].groups()
            args = [eval(x) for x in args.split(', ')]
            result = known_actions[action](*args)
            next_prompt = "Observation: {}".format(result)

In [42]:
question = "What are the simple and compound interest on a loan of 1000 USD at 5% for 10 years?"

In [43]:
query_cbot(question)

Thought: I need to calculate both the simple and compound interest. I'll start with the simple interest using the calculate_si function.
Action: calculate_si: 1000, 5, 10
PAUSE
Thought: Now that I have the simple interest, I need to calculate the compound interest using the calculate_ci function.
Action: calculate_ci: 1000, 5, 10
PAUSE
Answer: The simple interest is 500.0 USD and the compound interest is approximately 628.89 USD.
Execution Complete
